# Import lib

In [1]:

from openai import OpenAI
import pandas as pd
import json
from sklearn.preprocessing import MultiLabelBinarizer
import time
import customInput

In [2]:
client = OpenAI()

In [3]:
def chat(prompt, topic):
    completion = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "user", "content":f"""{prompt}{topic}."""}
          ]
    )
    return completion.choices[0].message.content


In [2]:
equipment = pd.read_csv("data/equipment.csv")

In [3]:
equipment.head()

,id,equipment,number,capacity,muscle_groups,remark
0,0,treadmill,9,1,NaN,NaN
1,1,rowing machine,4,1,NaN,NaN
2,2,upright bike,2,1,NaN,NaN
3,3,elliptical machine,3,1,NaN,NaN
4,4,bench,7,1,NaN,NaN


In [ ]:
equipment.iloc

# Test input format

In [4]:
df = pd.read_csv("data/equipment.csv")

In [5]:
numAvailable, useTime = customInput.getUsageList(df)

In [6]:
numAvailable

{0: 9,
 1: 3,
 2: 2,
 3: 3,
 4: 0,
 5: 1,
 6: 0,
 7: 0,
 8: 0,
 9: 1,
 10: 0,
 11: 0,
 12: 1,
 13: 0,
 14: 1,
 15: 0,
 16: 0,
 17: 1,
 18: 2,
 19: 1,
 20: 0,
 21: 0,
 22: 1,
 23: 0,
 24: 0,
 25: 1,
 26: 1,
 27: 1,
 28: 2,
 29: 9,
 30: 1,
 31: 3,
 32: 1,
 33: 1,
 34: 0,
 35: 1,
 36: 0,
 37: 1,
 38: 0,
 39: 4,
 40: 2}

In [7]:
useTime

{0: [],
 1: [20],
 2: [],
 3: [],
 4: [14, 29, 17, 25, 20, 13, 8],
 5: [],
 6: [20],
 7: [18],
 8: [27],
 9: [10, 17, 20, 16],
 10: [27],
 11: [17, 15],
 12: [],
 13: [23, 10],
 14: [],
 15: [11],
 16: [11],
 17: [22, 25, 27],
 18: [28],
 19: [],
 20: [14],
 21: [7],
 22: [],
 23: [14],
 24: [15],
 25: [],
 26: [6],
 27: [],
 28: [29, 16],
 29: [7, 28, 20],
 30: [25, 7, 24, 19],
 31: [],
 32: [],
 33: [],
 34: [19],
 35: [],
 36: [18, 29, 11],
 37: [],
 38: [14],
 39: [15, 21, 9, 17],
 40: []}

In [8]:
numWait, waitTime = customInput.getWaitingList(numAvailable, df)

In [9]:
waitTime

{0: [],
 1: [],
 2: [],
 3: [],
 4: [13, 16, 10, 7],
 5: [],
 6: [11, 26],
 7: [16, 23, 18, 7, 12],
 8: [],
 9: [],
 10: [19, 12, 6],
 11: [26, 11],
 12: [],
 13: [19, 11, 5],
 14: [],
 15: [26, 21, 9, 5],
 16: [5, 13, 29, 14, 24],
 17: [],
 18: [],
 19: [],
 20: [23, 6],
 21: [7, 25, 18],
 22: [],
 23: [24, 21, 25],
 24: [15, 17],
 25: [],
 26: [],
 27: [],
 28: [],
 29: [],
 30: [],
 31: [],
 32: [],
 33: [],
 34: [29, 26, 28, 20, 14],
 35: [],
 36: [10, 29, 8, 27, 5],
 37: [],
 38: [9, 11, 28, 7],
 39: [],
 40: []}

In [10]:
numWait

{0: 0,
 1: 0,
 2: 0,
 3: 0,
 4: 4,
 5: 0,
 6: 2,
 7: 5,
 8: 0,
 9: 0,
 10: 3,
 11: 2,
 12: 0,
 13: 3,
 14: 0,
 15: 4,
 16: 5,
 17: 0,
 18: 0,
 19: 0,
 20: 2,
 21: 3,
 22: 0,
 23: 3,
 24: 2,
 25: 0,
 26: 0,
 27: 0,
 28: 0,
 29: 0,
 30: 0,
 31: 0,
 32: 0,
 33: 0,
 34: 5,
 35: 0,
 36: 5,
 37: 0,
 38: 4,
 39: 0,
 40: 0}

In [13]:
for i in range(len(numAvailable)):
  if numAvailable[i] == df.loc[i]["number"] * df.loc[i]["capacity"]:
    print(df.loc[i]['equipment'], numAvailable[i], numWait[i])

elliptical machine 3 0
standing calf 1 0
abdominal machine 1 0
multi press machine 1 0
inner/outer thigh 2 0
leg press/calf extension 1 0
functional trainer 4 0
preacher curl slate 1 0
leg curl 1 0
lat pulldown 1 0
booty builder 7 1 0
free standing space saving modular rig 3 0
incline chest press 3 0
seated row 1 0
puzzle mat 8 0


In [14]:
df["availabel number"] = numAvailable
df["expected usage time for occupied equipments"] = useTime
df["waiting number"] = numWait
df["expected usage time for waiting people"] = waitTime

In [15]:
df.head()

,id,equipment,number,capacity,muscle_groups,remark,availabel number,expected usage time for occupied equipments,waiting number,expected usage time for waiting people
0,0,treadmill,9,1,NaN,NaN,2,"[8, 10, 11, 15, 18, 9, 24]",5,"[5, 23, 27, 20, 7]"
1,1,rowing machine,4,1,NaN,NaN,2,"[26, 12]",5,"[6, 7, 10, 17, 12]"
2,2,upright bike,2,1,NaN,NaN,1,[7],4,"[24, 15, 9, 21]"
3,3,elliptical machine,3,1,NaN,NaN,3,[],0,[]
4,4,bench,7,1,NaN,NaN,0,"[15, 11, 20, 13, 12, 22, 17]",1,[11]


# Get muscle groups using ChatGPT

In [4]:
get_muscle = """
You are a professional exercise physiologist, expertise in workout knowledge and human body muscle groups. \
Let's proceed step by step. \
Kindly furnish a list listing every possible muscle groups involved. \
Organize the list from large to small, based on the provided equipment. \
Please conclude the file with the marker [end]. \
For example,
Muscle Group:
Hamstrings
Gluteus Maximus
Erector Spinae
Gastrocnemius
Soleus
Adductor Magnus
Semimembranosus
Semitendinosus
Biceps Femoris
Popliteus
Gracilis
[end]

Now, given equipment

"""

In [87]:
test = chat(get_muscle, "leg curl")

In [89]:
test

'Muscle Group:\nHamstrings\nBiceps Femoris\nSemimembranosus\nSemitendinosus\nGracilis\nAdductor Magnus\nPopliteus\nGastrocnemius\nSoleus\nGluteus Maximus\nErector Spinae\n[end]'

In [91]:
sp = test.split("\n")
sp

['Muscle Group:',
 'Hamstrings',
 'Biceps Femoris',
 'Semimembranosus',
 'Semitendinosus',
 'Gracilis',
 'Adductor Magnus',
 'Popliteus',
 'Gastrocnemius',
 'Soleus',
 'Gluteus Maximus',
 'Erector Spinae',
 '[end]']

In [8]:
muscle_groups = []

```{python}
# count = 0
for item in equipment.loc[:, 'equipment']:
  # if count < 5:
    muscle_groups.append(getMuscleGroup(item))
    # count += 1
```

```{python}
with open("data/muscle_groups_ori.txt", "w") as f:
  for t in muscle_groups:
    f.write(f"{t}\n")
```

In [ ]:
muscle_groups

# Data Processing

### Counting
Count the number of all muscle groups and translate the names into Traditional Chinese.

In [52]:
muscle_groups_all = []
muscle_dict = dict()
for i in range(len(muscle_groups)):
  # num_split = 0
  # num_group = 0
  muscles = []
  # print(i)
  for muscle in muscle_groups[i].split("\n"):
    # num_split += 1
    if not (muscle.startswith("Muscle Group") or muscle == ''):
      if muscle == "[end]":
        # print(f"END:DDD, {muscle}")
        break
      # num_group += 1
      if muscle.split(". ")[0].isdigit():
        muscle = muscle.split(". ")[1]
      elif muscle.startswith("- "):
        muscle = muscle.split("- ")[1]
      # print(muscle)
      muscles.append(muscle)
      muscle_groups_all.append(muscle)
  muscle_dict[i] = muscles

In [53]:
len(muscle_groups_all)

485

```{python}
# Convert and write JSON object to file
with open("data/muscle_dict.json", "w") as outfile:
    json.dump(muscle_dict, outfile)
```

In [122]:
with open("data/muscle_dict.json", "r") as f:
  muscle_dict = json.load(f)

In [ ]:
muscle_dict

In [131]:
# flatten the lists and get unique items
unique_items = list(set(item for sublist in muscle_dict.values() for item in sublist))

In [133]:
unique_items.sort()

In [134]:
unique_items

['Adductor Brevis',
 'Adductor Longus',
 'Adductor Magnus',
 'Anconeus',
 'Anterior Deltoid',
 'Anterior Deltoids (Front shoulder)',
 'Biceps',
 'Biceps Brachii',
 'Biceps Femoris',
 'Biceps brachii',
 'Brachialis',
 'Brachioradialis',
 'Calf Muscles',
 'Calves',
 'Calves (Gastrocnemius and Soleus)',
 'Calves (Gastrocnemius, Soleus)',
 'Coracobrachialis',
 'Deltoid',
 'Deltoids',
 'Deltoids (Anterior, Lateral, and Posterior)',
 'Deltoids (Anterior, Middle, Posterior)',
 'Deltoids (Posterior)',
 'Deltoids (Rear)',
 'Erector Spinae',
 'Erector Spinae (Spinalis)',
 'Extensor Digitorum Longus',
 'Extensor Hallucis Longus',
 'External Obliques',
 'Flexor Digitorum Longus',
 'Flexor Hallucis Longus',
 'Forearm muscles (Flexors and Extensors)',
 'Gastrocnemius',
 'Gastrocnemius (Lateral head)',
 'Gastrocnemius (Medial head)',
 'Gluteus Maximus',
 'Gluteus Medius',
 'Gluteus Minimus',
 'Gracilis',
 'Hamstrings',
 'Hamstrings (Biceps Femoris, Semitendinosus, Semimembranosus)',
 'Hip Abductors',

| Original | Replaced |
| --- | --- |
| Anterior Deltoid | Anterior Deltoids (Front shoulder) |
| Biceps | Biceps Brachii |
| Biceps brachii | Biceps Brachii |
| Calves | Calves (Gastrocnemius and Soleus) |
| Calves (Gastrocnemius, Soleus) | Calves (Gastrocnemius and Soleus) |
| Deltoid | Deltoids (Anterior, Lateral, and Posterior) |
| Deltoids | Deltoids (Anterior, Lateral, and Posterior)
| Deltoids (Anterior, Middle, Posterior) | Deltoids (Anterior, Lateral, and Posterior) |
| Hamstrings | Hamstrings (Biceps Femoris, Semitendinosus, Semimembranosus) |
| Rear Deltoids | Posterior Deltoid |
| Deltoids (Posterior) | Posterior Deltoid |
| Deltoids (Rear) | Posterior Deltoid |
| Medial Deltoid | Lateral Deltoid |
| Pectoralis Major (Upper chest) | Pectoralis Major (Clavicular Head) |
| Serratus Anterior | Serratus Anterior (Upper side of ribs) |
| Terres Major | Teres Major |
| Transversus Abdominis | Transverse Abdominis |
| Triceps | Triceps Brachii (Back of upper arm) |
| Triceps Brachii | Triceps Brachii (Back of upper arm) |
| Trapezius (Middle and Lower) | Trapezius (middle and lower fibers) |
| Quadriceps | Quadriceps (Rectus Femoris, Vastus Lateralis, Vastus Medialis, Vastus Intermedius) |
| Rhomboideus | Rhomboids |

### Replacement

In [184]:
muscle_dict_rep = muscle_dict.copy()

In [185]:
# replacement table
replacement_table = {
    'Anterior Deltoid': 'Anterior Deltoids (Front shoulder)',
    'Biceps': 'Biceps Brachii',
    'Biceps brachii': 'Biceps Brachii',
    'Calves': 'Calves (Gastrocnemius and Soleus)',
    'Calves (Gastrocnemius, Soleus)': 'Calves (Gastrocnemius and Soleus)',
    'Deltoid': 'Deltoids (Anterior, Lateral, and Posterior)',
    'Deltoids': 'Deltoids (Anterior, Lateral, and Posterior)',
    'Deltoids (Anterior, Middle, Posterior)': 'Deltoids (Anterior, Lateral, and Posterior)',
    'Hamstrings': 'Hamstrings (Biceps Femoris, Semitendinosus, Semimembranosus)',
    'Rear Deltoids': 'Posterior Deltoid',
    'Deltoids (Posterior)': 'Posterior Deltoid',
    'Deltoids (Rear)': 'Posterior Deltoid',
    'Medial Deltoid': 'Lateral Deltoid',
    'Pectoralis Major (Upper chest)': 'Pectoralis Major (Clavicular Head)',
    'Serratus Anterior': 'Serratus Anterior (Upper side of ribs)',
    'Terres Major': 'Teres Major',
    'Transversus Abdominis': 'Transverse Abdominis',
    'Triceps': 'Triceps Brachii (Back of upper arm)',
    'Triceps Brachii': 'Triceps Brachii (Back of upper arm)',
    'Trapezius (Middle and Lower)': 'Trapezius (middle and lower fibers)',
    'Quadriceps': 'Quadriceps (Rectus Femoris, Vastus Lateralis, Vastus Medialis, Vastus Intermedius)',
    'Rhomboideus': 'Rhomboids'
}

# function to replace values in a list using a mapping
def replaceValues(muscle_list, replacement_mapping):
    rep = [replacement_mapping.get(muscle, muscle) for muscle in muscle_list]
    return rep
    # return [print(replacement_mapping.get(muscle, muscle)) for muscle in muscle_list]

# Apply the replacement to each list in the dictionary
for key, muscles in muscle_dict_rep.items():
    # print(key)
    muscle_dict_rep[key] = replaceValues(muscles, replacement_table)

In [ ]:
muscle_dict_rep


#### Convert and write JSON object to file
with open("data/muscle_dict_rep.json", "w") as outfile:
    json.dump(muscle_dict_rep, outfile)


In [144]:
with open("data/muscle_dict_rep.json", "r") as f:
  muscle_dict_rep = json.load(f)

In [187]:
unique_items = list(set(item for sublist in muscle_dict_rep.values() for item in sublist))
unique_items.sort()

In [197]:
len(unique_items)

81

In [188]:
unique_items

['Adductor Brevis',
 'Adductor Longus',
 'Adductor Magnus',
 'Anconeus',
 'Anterior Deltoids (Front shoulder)',
 'Biceps Brachii',
 'Biceps Femoris',
 'Brachialis',
 'Brachioradialis',
 'Calf Muscles',
 'Calves (Gastrocnemius and Soleus)',
 'Coracobrachialis',
 'Deltoids (Anterior, Lateral, and Posterior)',
 'Erector Spinae',
 'Erector Spinae (Spinalis)',
 'Extensor Digitorum Longus',
 'Extensor Hallucis Longus',
 'External Obliques',
 'Flexor Digitorum Longus',
 'Flexor Hallucis Longus',
 'Forearm muscles (Flexors and Extensors)',
 'Gastrocnemius',
 'Gastrocnemius (Lateral head)',
 'Gastrocnemius (Medial head)',
 'Gluteus Maximus',
 'Gluteus Medius',
 'Gluteus Minimus',
 'Gracilis',
 'Hamstrings (Biceps Femoris, Semitendinosus, Semimembranosus)',
 'Hip Abductors',
 'Hip Adductors',
 'Hip Flexors',
 'Iliopsoas',
 'Infraspinatus',
 'Internal Obliques',
 'Internal and External Intercostals',
 'Internal and External Obliques',
 'Lateral Deltoid',
 'Latissimus Dorsi',
 'Levator Scapulae',


### Adding

In [182]:
# muscle_dict_rep_add = muscle_dict_rep.copy()

In [183]:
"""# adding table
add_table = {
    'Hamstrings (Biceps Femoris, Semitendinosus, Semimembranosus)': 'Hamstrings',
    'Pectoralis Major (Clavicular Head)': 'Pectoralis Major',
    'Pectoralis Major (Sternal Head)': 'Pectoralis Major',
    'Pectoralis Major (Upper chest)': 'Pectoralis Major',
    'Quadriceps (Rectus Femoris, Vastus Lateralis, Vastus Medialis, Vastus Intermedius)': 'Quadriceps',
    'Rhomboids': 'Rhomboideus',
    'Rhomboideus': 'Rhomboids',
    'Serratus Anterior (Upper side of ribs)': 'Serratus Anterior',
    'Trapezius (Middle and Lower)': 'Trapezius',
    'Trapezius (lower fibers)': 'Trapezius',
    'Trapezius (middle and lower fibers)': 'Trapezius',
    'Triceps Brachii (Back of upper arm)': 'Triceps Brachii',
    'Triceps Brachii (Long Head)': 'Triceps Brachii',
}

# update values in a list using a mapping
for key, muscles in muscle_dict_rep_add.items():
    # print(key)
    add_list = []
    for muscle in muscles:
        add = add_table.get(muscle)
        if add is not None:
            add_list.append(add)
    # print(add_list)
    if add_list != []:
        muscle_dict_rep_add[key] += add_list"""

In [ ]:
# muscle_dict_rep_add

In [ ]:
# Convert and write JSON object to file
# with open("data/muscle_dict_rep_add.json", "w") as outfile:
#     json.dump(muscle_dict_rep_add, outfile)

In [6]:
# with open("data/muscle_dict_rep_add.json", "r") as f:
#   muscle_dict_rep_add = json.load(f)

In [21]:
# flatten the lists and get unique items
# unique_items = list(set(item for sublist in muscle_dict_rep_add.values() for item in sublist))

In [ ]:
# unique_items

In [ ]:
# len(unique_items)

### Categorization

In [23]:
categorize ="""
You are a professional exercise physiologist, expertise in workout knowledge and human body muscle groups. \
Given 6 categories: arm, abdominal, shoulder, chest, back, and leg. \
Provide the category only. \
Example 1:
Rhomboid Minor
Back

Example 2:
Anterior Deltoids
Shoulder

Now, Categorize the following human muscle group:
"""

In [193]:
test = chat(categorize, unique_items[0])

In [194]:
test

'Leg'

In [ ]:
category = dict()
count = 0
for name in unique_items:
  # if count < 5:
    category[name] = chat(categorize, name)
    # count += 1

In [26]:
category

{'Adductor Brevis': 'Leg',
 'Adductor Longus': 'Leg',
 'Adductor Magnus': 'Leg',
 'Anconeus': 'Arm',
 'Anterior Deltoids (Front shoulder)': 'Shoulder',
 'Biceps Brachii': 'Arm',
 'Biceps Femoris': 'Leg',
 'Brachialis': 'Arm',
 'Brachioradialis': 'Arm',
 'Calf Muscles': 'Leg',
 'Calves (Gastrocnemius and Soleus)': 'Leg',
 'Coracobrachialis': 'Arm',
 'Deltoids (Anterior, Lateral, and Posterior)': 'Shoulder',
 'Erector Spinae': 'Back',
 'Erector Spinae (Spinalis)': 'Back',
 'Extensor Digitorum Longus': 'Leg',
 'Extensor Hallucis Longus': 'Leg',
 'External Obliques': 'Abdominal',
 'Flexor Digitorum Longus': 'Leg',
 'Flexor Hallucis Longus': 'Leg',
 'Forearm muscles (Flexors and Extensors)': 'Arm',
 'Gastrocnemius': 'Leg',
 'Gastrocnemius (Lateral head)': 'Leg',
 'Gastrocnemius (Medial head)': 'Leg',
 'Gluteus Maximus': 'Leg',
 'Gluteus Medius': 'Leg',
 'Gluteus Minimus': 'Leg',
 'Gracilis': 'Leg',
 'Hamstrings (Biceps Femoris, Semitendinosus, Semimembranosus)': 'Leg',
 'Hip Abductors': 'Le

In [198]:
# Convert and write JSON object to file
with open("data/category.json", "w") as outfile:
    json.dump(category, outfile)

In [25]:
with open("data/category.json", "r") as f:
  category = json.load(f)

In [204]:
def reverseDict(dic, value):
  v_list =[]
  for key in dic:
    if dic[key] == value:
      v_list.append(key)
  return v_list

In [205]:
category_rev = dict()

In [206]:
for c in category.values():
  c_list = reverseDict(category, c)
  category_rev[c] = c_list

In [207]:
category_rev

{'Leg': ['Adductor Brevis',
  'Adductor Longus',
  'Adductor Magnus',
  'Biceps Femoris',
  'Calf Muscles',
  'Calves (Gastrocnemius and Soleus)',
  'Extensor Digitorum Longus',
  'Extensor Hallucis Longus',
  'Flexor Digitorum Longus',
  'Flexor Hallucis Longus',
  'Gastrocnemius',
  'Gastrocnemius (Lateral head)',
  'Gastrocnemius (Medial head)',
  'Gluteus Maximus',
  'Gluteus Medius',
  'Gluteus Minimus',
  'Gracilis',
  'Hamstrings (Biceps Femoris, Semitendinosus, Semimembranosus)',
  'Hip Abductors',
  'Hip Adductors',
  'Hip Flexors',
  'Iliopsoas',
  'Peroneals',
  'Peroneus Brevis',
  'Peroneus Longus',
  'Peroneus Tertius',
  'Plantaris',
  'Popliteus',
  'Quadriceps (Rectus Femoris, Vastus Lateralis, Vastus Medialis, Vastus Intermedius)',
  'Rectus Femoris',
  'Sartorius',
  'Semimembranosus',
  'Semitendinosus',
  'Soleus',
  'Tensor Fasciae Latae',
  'Tibialis Anterior',
  'Vastus Intermedius',
  'Vastus Lateralis',
  'Vastus Medialis'],
 'Arm': ['Anconeus',
  'Biceps Brac

In [208]:
with open("data/category_rev.json", "w") as outfile:
    json.dump(category_rev, outfile)

Add categories to `muscle_dict_rep_add`

In [209]:
muscle_dict_comple = muscle_dict_rep.copy()

In [210]:
def addCategory(muscles):
  add_list = []
  for muscle in muscles:
    if category[muscle] not in add_list:
      add_list.append(category[muscle])
  return add_list

In [211]:
for key, muscles in muscle_dict_comple.items():
  add_list = addCategory(muscles)
  if add_list != []:
    muscle_dict_comple[key] += add_list

In [ ]:
muscle_dict_comple

In [213]:
with open("data/muscle_dict_comple.json", "w") as outfile:
    json.dump(muscle_dict_comple, outfile)

In [17]:
with open("data/muscle_dict_comple.json", "r") as f:
  muscle_dict_comple = json.load(f)

In [18]:
unique_items = list(set(item for sublist in muscle_dict_comple.values() for item in sublist))
unique_items.sort()

In [216]:
len(unique_items)

87

In [19]:
unique_items

['Abdominal',
 'Adductor Brevis',
 'Adductor Longus',
 'Adductor Magnus',
 'Anconeus',
 'Anterior Deltoids (Front shoulder)',
 'Arm',
 'Back',
 'Biceps Brachii',
 'Biceps Femoris',
 'Brachialis',
 'Brachioradialis',
 'Calf Muscles',
 'Calves (Gastrocnemius and Soleus)',
 'Chest',
 'Coracobrachialis',
 'Deltoids (Anterior, Lateral, and Posterior)',
 'Erector Spinae',
 'Erector Spinae (Spinalis)',
 'Extensor Digitorum Longus',
 'Extensor Hallucis Longus',
 'External Obliques',
 'Flexor Digitorum Longus',
 'Flexor Hallucis Longus',
 'Forearm muscles (Flexors and Extensors)',
 'Gastrocnemius',
 'Gastrocnemius (Lateral head)',
 'Gastrocnemius (Medial head)',
 'Gluteus Maximus',
 'Gluteus Medius',
 'Gluteus Minimus',
 'Gracilis',
 'Hamstrings (Biceps Femoris, Semitendinosus, Semimembranosus)',
 'Hip Abductors',
 'Hip Adductors',
 'Hip Flexors',
 'Iliopsoas',
 'Infraspinatus',
 'Internal Obliques',
 'Internal and External Intercostals',
 'Internal and External Obliques',
 'Lateral Deltoid',
 

## Translation
EN -> TC

In [20]:
traslate ="""
You are a professional exercise physiologist, expertise in workout knowledge and human body muscle groups. \
Translate the following muscle group into TRADITIONAL Chinese (Taiwan). \
Exclude phonetic symbols. \
Simply provide the translated word. \
Example 1:
Biceps
二頭肌

Example 2:
Biceps Femoris
股二頭肌

Now, translate this muscle group:
"""

In [27]:
len(unique_items)

87

In [219]:
test = chat(traslate, unique_items[0])

In [220]:
test

'腹肌'

In [ ]:
translate_C2E_dict = dict()
count = 0
for name in unique_items:
  translate_C2E_dict[chat(traslate, name)] = name

In [75]:
len(translate_C2E_dict)

87

In [ ]:
translate_C2E_dict

In [71]:
# add missing muscles
translate_C2E_dict[chat(traslate, "Pectoralis Major (Clavicular Head)")] = "Pectoralis Major (Clavicular Head)"

In [76]:
with open("data/translate_C2E_dict.json", "w") as outfile:
    json.dump(translate_C2E_dict, outfile)

In [40]:
with open("data/translate_C2E_dict.json", "r") as f:
    translate_C2E_dict = json.load(f)

In [73]:
translate_E2C_dict = {
  value: key for key, value in translate_C2E_dict.items()
}

In [74]:
with open("data/translate_E2C_dict.json", "w") as outfile:
    json.dump(translate_E2C_dict, outfile)

## Muscle groups list in Chinese

In [3]:
translate_E2C_dict = {}
category_dict = {}
with open("data/translate_E2C_dict.json", "r") as f:
  translate_E2C_dict = json.load(f)
with open("data/category_rev.json", "r") as f:
  category_dict = json.load(f)

In [6]:
category_cn = {}

for cat in category_dict.keys():
  category_cn[translate_E2C_dict[cat]] = [ translate_E2C_dict[muscle] for muscle in category_dict[cat]]

In [ ]:
category_cn

In [9]:
with open("data/category_cn.json", "w") as outfile:
  json.dump(category_cn, outfile)

# Update equipment.csv
Update muscle_groups info

## list

In [28]:
equipment_list = equipment.copy()

In [26]:
list(muscle_dict_comple.values())

[['Quadriceps (Rectus Femoris, Vastus Lateralis, Vastus Medialis, Vastus Intermedius)',
  'Hamstrings (Biceps Femoris, Semitendinosus, Semimembranosus)',
  'Gluteus Maximus',
  'Gastrocnemius',
  'Soleus',
  'Iliopsoas',
  'Tensor Fasciae Latae',
  'Adductor Magnus',
  'Hip Flexors',
  'Leg'],
 ['Quadriceps (Rectus Femoris, Vastus Lateralis, Vastus Medialis, Vastus Intermedius)',
  'Hamstrings (Biceps Femoris, Semitendinosus, Semimembranosus)',
  'Gluteus Maximus',
  'Erector Spinae',
  'Gastrocnemius',
  'Soleus',
  'Adductor Magnus',
  'Tibialis Anterior',
  'Biceps Brachii',
  'Brachialis',
  'Rhomboids',
  'Trapezius',
  'Posterior Deltoid',
  'Leg',
  'Back',
  'Arm',
  'Shoulder'],
 ['Quadriceps (Rectus Femoris, Vastus Lateralis, Vastus Medialis, Vastus Intermedius)',
  'Hamstrings (Biceps Femoris, Semitendinosus, Semimembranosus)',
  'Gluteus Maximus',
  'Gastrocnemius',
  'Soleus',
  'Leg'],
 ['Quadriceps (Rectus Femoris, Vastus Lateralis, Vastus Medialis, Vastus Intermedius)',

In [29]:
equipment_list['muscle_groups'] = list(muscle_dict_comple.values())

In [34]:
equipment_list.head()

,id,equipment,number,capacity,muscle_groups,remark
0,0,treadmill,9,1,"[Quadriceps (Rectus Femoris, Vastus Lateralis,...",NaN
1,1,rowing machine,4,1,"[Quadriceps (Rectus Femoris, Vastus Lateralis,...",NaN
2,2,upright bike,2,1,"[Quadriceps (Rectus Femoris, Vastus Lateralis,...",NaN
3,3,elliptical machine,3,1,"[Quadriceps (Rectus Femoris, Vastus Lateralis,...",NaN
4,4,bench,7,1,"[Quadriceps (Rectus Femoris, Vastus Lateralis,...",NaN


In [36]:
equipment_list.to_csv("data/equipment_list.csv", index=False)

## str

In [37]:
equipment_str = equipment.copy()

In [39]:
equipment_str['muscle_groups'] = equipment_list['muscle_groups'].apply(lambda x: ','.join(map(str, x)))

In [41]:
equipment_str.head()

,id,equipment,number,capacity,muscle_groups,remark
0,0,treadmill,9,1,"Quadriceps (Rectus Femoris, Vastus Lateralis, ...",NaN
1,1,rowing machine,4,1,"Quadriceps (Rectus Femoris, Vastus Lateralis, ...",NaN
2,2,upright bike,2,1,"Quadriceps (Rectus Femoris, Vastus Lateralis, ...",NaN
3,3,elliptical machine,3,1,"Quadriceps (Rectus Femoris, Vastus Lateralis, ...",NaN
4,4,bench,7,1,"Quadriceps (Rectus Femoris, Vastus Lateralis, ...",NaN


In [42]:
equipment_str.to_csv("data/equipment_str.csv", index=False)

## One-Hot

In [69]:
equipment_onehot = equipment.iloc[:, :-2].copy()

In [48]:
mlb = MultiLabelBinarizer()
res = pd.DataFrame(mlb.fit_transform(equipment_list['muscle_groups']),
                   columns=mlb.classes_,
                   index=equipment_list['muscle_groups'].index)

In [50]:
res.head()

,Abdominal,Adductor Brevis,Adductor Longus,Adductor Magnus,Anconeus,Anterior Deltoids (Front shoulder),Arm,Back,Biceps Brachii,Biceps Femoris,...,Tibialis Anterior,Transverse Abdominis,Trapezius,Trapezius (lower fibers),Trapezius (middle and lower fibers),Triceps Brachii (Back of upper arm),Triceps Brachii (Long Head),Vastus Intermedius,Vastus Lateralis,Vastus Medialis
0,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,1,0,0,1,1,1,0,...,1,0,1,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,0,0,1,0,0,1,1,1,0,...,1,0,1,0,0,1,0,0,0,0
4,0,0,0,0,0,0,1,1,1,0,...,0,0,1,0,0,1,0,0,0,0


In [70]:
equipment_onehot = equipment_onehot.join(res).join(equipment['remark'])

In [71]:
equipment_onehot.head()

,id,equipment,number,capacity,Abdominal,Adductor Brevis,Adductor Longus,Adductor Magnus,Anconeus,Anterior Deltoids (Front shoulder),...,Transverse Abdominis,Trapezius,Trapezius (lower fibers),Trapezius (middle and lower fibers),Triceps Brachii (Back of upper arm),Triceps Brachii (Long Head),Vastus Intermedius,Vastus Lateralis,Vastus Medialis,remark
0,0,treadmill,9,1,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,NaN
1,1,rowing machine,4,1,0,0,0,1,0,0,...,0,1,0,0,0,0,0,0,0,NaN
2,2,upright bike,2,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,NaN
3,3,elliptical machine,3,1,1,0,0,1,0,0,...,0,1,0,0,1,0,0,0,0,NaN
4,4,bench,7,1,0,0,0,0,0,0,...,0,1,0,0,1,0,0,0,0,NaN


In [72]:
equipment_onehot.to_csv("data/equipment_onehot.csv", index=False)